<a href="https://colab.research.google.com/github/rajasreekalli/Data-Visualization/blob/main/A_L_E_Core.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
import pandas as pd
import numpy as np

In [146]:
df = pd.read_csv('/content/drive/MyDrive/abalone.data')
header=None

In [147]:
df

,M,0.455,0.365,0.095,0.514,0.2245,0.101,0.15,15
0,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
1,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
2,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
3,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
4,I,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.1200,8
...,...,...,...,...,...,...,...,...,...
4171,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4172,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4173,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4174,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [148]:
df.columns = ['Length', 'Diameter', 'Height', 'whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight', 'Sex', 'Rings']

In [149]:
df

,Length,Diameter,Height,whole weight,Shucked weight,Viscera weight,Shell weight,Sex,Rings
0,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
1,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
2,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
3,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
4,I,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.1200,8
...,...,...,...,...,...,...,...,...,...
4171,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4172,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4173,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4174,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [150]:
 df.isnull().sum()

Length            0
Diameter          0
Height            0
whole weight      0
Shucked weight    0
Viscera weight    0
Shell weight      0
Sex               0
Rings             0
dtype: int64

In [151]:
# 1)Separate your data into the features matrix (X) and target vector (y).
y = df['Rings']
X = df.drop(columns = ['Rings', 'Sex'])

In [152]:
# 2) Create a train, test, split on the data. Please use random number 42 for consistency.
from sklearn.model_selection import train_test_split

In [153]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [154]:
# 3) Make a pipeline to include the 1) column transformations and  a 2) and your linear regression model.
#For the column transformations:a) Use column_selector to select the categorical columns and the numerical columns
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OneHotEncoder

In [155]:
# The categorical selector was made in the last lesson
cat_selector = make_column_selector(dtype_include='object')
# make a numeric column selector
num_selector = make_column_selector(dtype_include='number')

In [156]:
median_imputer = SimpleImputer(strategy='median')
freq_imputer = SimpleImputer(strategy = 'most_frequent')
one_hot_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

In [157]:
#make tuples of a preprocessing object and a selector
#impute median of numeric columns
median_tuple = (median_imputer, num_selector)
#impute most frequent of categorical columns
freq_tuple = (freq_imputer, cat_selector)
#one-hot encode categorical columns
ohe_tuple = (one_hot_encoder, cat_selector)

In [158]:
from sklearn.compose import make_column_transformer
#instantiate column transformer
#give it the steps in the order to apply them.
column_transformer = make_column_transformer(median_tuple, freq_tuple, ohe_tuple)

In [159]:
column_transformer.fit(X_train)

ColumnTransformer(transformers=[('simpleimputer-1',
                                 SimpleImputer(strategy='median'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fda10e0f290>),
                                ('simpleimputer-2',
                                 SimpleImputer(strategy='most_frequent'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fda10e0f390>),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fda10e0f390>)])

In [160]:
X_train_processed = column_transformer.transform(X_train)
X_test_processed = column_transformer.transform(X_test)
X_train_processed

array([[0.34, 0.255, 0.095, ..., 0.0, 0.0, 1.0],
       [0.62, 0.46, 0.17, ..., 0.0, 0.0, 1.0],
       [0.555, 0.44, 0.155, ..., 1.0, 0.0, 0.0],
       ...,
       [0.52, 0.43, 0.15, ..., 1.0, 0.0, 0.0],
       [0.575, 0.46, 0.15, ..., 1.0, 0.0, 0.0],
       [0.595, 0.465, 0.14, ..., 1.0, 0.0, 0.0]], dtype=object)

In [161]:
# instantiate imputer and regression estimator
imputer = SimpleImputer(strategy='mean')
lin_reg = LinearRegression()
# combine into a pipeline
pipe = make_pipeline(imputer, lin_reg)

In [162]:
from sklearn.compose import make_column_selector
from sklearn.impute import SimpleImputer

In [163]:
Length = {'M': 0, 'F': 1, 'I': 2}
X_train['Length'] = X_train['Length'].replace(Length)

In [164]:
Length = {'M': 0, 'F': 1, 'I': 2}
X_test['Length'] = X_test['Length'].replace(Length)

In [165]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('linearregression', LinearRegression())])

In [166]:
print(f'Train R2: {r2_score(y_train, pipe.predict(X_train))}')
print(f'Test R2: {r2_score(y_test, pipe.predict(X_test))}')

Train R2: 0.5164909391131896
Test R2: 0.5585117410202428


In [167]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

In [168]:
reg.fit(X_train,y_train)

LinearRegression()

In [169]:
train_score = reg.score(X_train, y_train)
print(train_score)

0.5164909391131896


In [170]:
test_score = reg.score(X_test, y_test)
print(test_score)

0.5585117410202428
